# Imports and Preprocessing

In [69]:
import numpy as np
import pandas as pd
import tensorflow as tf
from scikeras.wrappers import KerasRegressor

In [70]:
import os
import pandas as pd
folder_path = 'Data_Test_v1'
file_names = ['data_test.csv', 'data_train.csv', 'target_test.csv', 'target_train.csv']

data_frames = []
for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)
    data_frames.append(df)

data_test = data_frames[0]
data_train = data_frames[1]
target_test = data_frames[2]    
target_train = data_frames[3]

# ANN Model

In [71]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.optimizers import Adam
from scipy.stats import reciprocal

# Define function to create the ANN model 
def create_model(n_hidden, n_neurons, learning_rate, activation, input_shape=[3]): 
    model = Sequential()
    model.add(Dense(n_neurons, activation="relu", input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
    model.add(Dense(1))
    optimizer = Adam() #learning_rate=learning_rate_input
    model.compile(loss="mean_squared_error", optimizer=optimizer)
    return model

# Define function to build the model
def model_builder(n_hidden=1, n_neurons=30, learning_rate=3e-3 , activation = 'relu'):
    return KerasRegressor(build_fn=create_model, verbose=0, n_hidden=n_hidden, n_neurons=n_neurons, learning_rate=learning_rate, activation=activation) 

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_hidden": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(1e-4, 1e-2),
    "batch_size": [16, 32, 64, 128],
    "epochs": [10, 20, 30, 40, 50],
    "activation": ['relu', 'sigmoid', 'tanh']
}

# Create the model and optimize with Randomized Search
model_ANN = model_builder()
rnd_search_cv_ANN = RandomizedSearchCV(model_ANN, param_distribs, n_iter=10, cv=3)

# Fit the model on the training data
rnd_search_cv_ANN.fit(data_train, target_train)
print(rnd_search_cv_ANN.best_params_)

# Predict the target values
target_pred_ANN = rnd_search_cv_ANN.predict(data_test)

c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\lanza\Integrated-vs-Seperated-Master-Thesis\.venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future relea

{'activation': 'sigmoid', 'batch_size': 16, 'epochs': 30, 'learning_rate': 0.0005728768318037444, 'n_hidden': 0, 'n_neurons': 32}


# DT using LightGBM

In [72]:
import lightgbm as lgb

# Create the model
model_DT = lgb.LGBMRegressor()

# Define the hyperparameters to search for the Randomized Search
param_distribs = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.5],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
}

# Create the model and optimize with Randomized Search
rnd_search_cv_DT = RandomizedSearchCV(model_DT, param_distribs, n_iter=10, cv=3)

# Fit the model on the training data
rnd_search_cv_DT.fit(data_train, target_train)
print(rnd_search_cv_DT.best_params_)

# Make predictions
target_pred_DT = rnd_search_cv_DT.predict(data_test)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000118 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 49
[LightGBM] [Info] Number of data points in the train set: 138, number of used features: 3
[LightGBM] [Info] Start training from score 1561.014493
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further spli

# Forecasting Error

In [73]:
# Calculate forecast error for ANN
train_pred_ANN = rnd_search_cv_ANN.predict(data_train)
train_pred_ANN_flatten = train_pred_ANN.flatten()
target_train = target_train.values.flatten()
forecast_error_ANN = target_train - train_pred_ANN_flatten

In [74]:
# Calculate forecast error for DT
train_pred_DT = rnd_search_cv_DT.predict(data_train)
train_pred_DT_flatten = train_pred_DT.flatten()
forecast_error_DT = target_train - train_pred_DT_flatten

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).


# Predicting Order Quantities 

In [75]:
# Initialize an empty list to store the final order quantities
final_order_quantities_ANN = []
final_order_quantities_DT = []

# Define costs
c = 2  # cost per unit of product
s = 1  # salvage value per unit of leftover product
p = 3  # price per unit of product sold

# Calculate critical ratio
critical_ratio = (p - c) / ((c - s) + (p - c))
print(critical_ratio)

0.5


In [76]:
from scipy.stats import norm
# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_ANN[i] + np.random.choice(forecast_error_ANN, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_ANN.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_ANN = np.array(final_order_quantities_ANN)

In [77]:
# Loop over each week in data_test
for i in range(len(data_test)):

    # Number of scenarios
    num_scenarios = 1000

    # Generate demand scenarios for the week
    demand_scenarios_single = target_pred_DT[i] + np.random.choice(forecast_error_DT, size=num_scenarios)

    # Solve newsvendor problem for the week
    order_quantity_scenarios = norm.ppf(critical_ratio, loc=demand_scenarios_single)

    # Append the order quantity to the list
    final_order_quantities_DT.append(np.mean(order_quantity_scenarios))

# Convert the list to a numpy array
final_order_quantities_DT = np.array(final_order_quantities_DT)

In [78]:
print(final_order_quantities_ANN)

[1520.91145459 1526.09370703 1548.85721667 1535.45467224 1536.45527502
 1524.96232312 1531.44496265 1541.64205933 1548.17489539 1537.45702991
 1527.81080566 1539.91797949 1525.92267993 1512.96529041 1523.16260681
 1535.1525791  1564.04379395 1541.93685095 1535.52017993 1540.24665625
 1533.01728308 1540.50715015 1545.31658459 1536.19121448 1537.39975439
 1533.98566211 1507.86013672 1521.70245996 1537.56504956 1537.90077197
 1534.29992517 1547.68470886 1556.94581665 1551.0238927  1527.27940674
 1524.50752087 1530.20725232 1527.33851831 1521.20841113 1523.73626782
 1541.87640527 1542.5158302  1536.03177197 1545.31248962 1536.38116589
 1528.04397632 1554.5582948  1524.67844946 1533.64663489 1523.27962012
 1545.17954626 1541.20779785 1524.34785205]


# Calculate Costs

In [79]:
# Loop over each week in target_test
overall_profit_ANN = 0
overall_profit_DT = 0
for i in range(len(target_test)):

    # Calculate the profit for the week (without over or understock costs)
    profit_ANN = (p - c) * round(final_order_quantities_ANN[i])
    profit_DT = (p - c) * round(final_order_quantities_DT[i])

    # Calculate understock and overstock costs
    overstock_ANN = 0
    overstock_DT = 0
    understock_ANN = 0
    understock_DT = 0
    if final_order_quantities_ANN[i] < target_test.values[i]:
        understock_ANN = (p - c) * (target_test.values[i] - round(final_order_quantities_ANN[i]))
    if final_order_quantities_ANN[i] > target_test.values[i]:
        overstock_ANN = (c - s) * (round(final_order_quantities_ANN[i]) - target_test.values[i])
    if final_order_quantities_DT[i] < target_test.values[i]:
        understock_DT = (p - c) * (target_test.values[i] - round(final_order_quantities_DT[i]))
    if final_order_quantities_DT[i] > target_test.values[i]:
        overstock_DT = (c - s) * (round(final_order_quantities_DT[i]) - target_test.values[i])
    

    # Calculate the total profit for the week
    total_profit_ANN = profit_ANN - overstock_ANN - understock_ANN
    total_profit_DT = profit_DT - overstock_DT - understock_DT

    # Add the total profit to the overall profit
    overall_profit_ANN += total_profit_ANN
    overall_profit_DT += total_profit_DT
   

# Print the overall profit
print('Overall profit for ANN: ', overall_profit_ANN)
print('Overstock: ', overstock_ANN, ' Understock: ', understock_ANN)
print('Overall profit for DT: ', overall_profit_DT)
print('Overstock: ', overstock_DT, ' Understock: ', understock_DT)

Overall profit for ANN:  [65805]
Overstock:  0  Understock:  [327]
Overall profit for DT:  [78705]
Overstock:  [7]  Understock:  0
